In [35]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [36]:
import pymc3 as pm

In [37]:
np.random.seed(23) # from random.org

# 3.1. Hierarchical GLAM estimation and out of sample prediction

## Load data

In [38]:
# Load data
sufix = '_Like_NoBin_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Like_NoBin_31.csv')
# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,0,2009,1.10,0.95,0.568396,0.431604
1,0,1,0,3371,2.00,1.70,0.762332,0.237668
2,0,2,1,1700,1.10,2.30,0.446809,0.553191
3,0,3,1,7466,1.25,1.40,0.532352,0.467648
4,0,4,1,1889,2.00,2.30,0.529736,0.470264


In [39]:
data.subject.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

## Split data in training and test sets

In [40]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


## Hierarchical GLAM estimation

### 1. full GLAM

In [9]:
# Fitting full GLAM
print('Fitting full GLAM hierarchically...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2018_full_hierarchical_cv'+sufix+'.npy')):
    glam_full.make_model('hierarchical', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_FF2018_full_hierarchical_cv'+sufix+'.npy'))   
# Save parameter estimates
np.save(str('results/estimates/glam_FF2018_full_hierarchical_cv'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

Fitting full GLAM hierarchically...
Generating hierarchical model for 31 subjects...


/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

Fitting 1 model(s) using NUTS...
  Fitting model 1 of 1...


/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [tau, tau_sd, tau_mu, SNR, SNR_sd, SNR_mu, gamma, gamma_sd, gamma_mu, v, v_sd, v_mu]
Sampling 2 chains:   0%|          | 0/6000 [00:00<?, ?draws/s]

/!\ Automatically setting parameter precision...


,SNR,SNR_mu,SNR_sd,b,gamma,gamma_mu,gamma_sd,p_error,s,t0,tau,tau_mu,tau_sd,v,v_mu,v_sd
0,151.23,164.29,35.54,1.0,-0.11,-0.26,0.38,0.05,0.007843,0.0,3.74,3.34,1.14,0.000047,0.000052,0.000013
1,137.71,164.29,35.54,1.0,0.29,-0.26,0.38,0.05,0.010344,0.0,4.30,3.34,1.14,0.000072,0.000052,0.000013
2,141.48,164.29,35.54,1.0,-0.00,-0.26,0.38,0.05,0.008574,0.0,2.59,3.34,1.14,0.000058,0.000052,0.000013
3,157.60,164.29,35.54,1.0,-0.10,-0.26,0.38,0.05,0.007178,0.0,4.65,3.34,1.14,0.000043,0.000052,0.000013
4,96.58,164.29,35.54,1.0,-0.37,-0.26,0.38,0.05,0.007063,0.0,4.22,3.34,1.14,0.000080,0.000052,0.000013
5,113.65,164.29,35.54,1.0,0.15,-0.26,0.38,0.05,0.008615,0.0,2.80,3.34,1.14,0.000059,0.000052,0.000013
6,163.65,164.29,35.54,1.0,-0.00,-0.26,0.38,0.05,0.010656,0.0,3.55,3.34,1.14,0.000069,0.000052,0.000013
7,150.49,164.29,35.54,1.0,-0.08,-0.26,0.38,0.05,0.007405,0.0,3.29,3.34,1.14,0.000045,0.000052,0.000013
8,175.26,164.29,35.54,1.0,-0.71,-0.26,0.38,0.05,0.009622,0.0,1.24,3.34,1.14,0.000053,0.000052,0.000013
9,137.66,164.29,35.54,1.0,-0.08,-0.26,0.38,0.05,0.007104,0.0,4.77,3.34,1.14,0.000048,0.000052,0.000013


In [10]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2018_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_FF2018_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_FF2018_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...


/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This coul

In [11]:
glam_full.waic

WAIC_r(WAIC=32704.32728053088, WAIC_se=0.0, p_WAIC=66.66452633644445, var_warn=1)

In [12]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2018_full'+ sufix +'.npy'), glam_full.loo)

/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/pymc3/stats.py:292: UserWarning: Estimated shape parameter of Pareto distribution is
        greater than 0.7 for one or more samples.
        You should consider using a more robust model, this is because
        importance sampling is less likely to work well if the marginal
        posterior and LOO posterior are very different. This is more likely to
        happen with a non-robust model and highly influential observations.
  happen with a non-robust model and highly influential observations.""")


In [13]:
glam_full.loo

LOO_r(LOO=32653.366046821335, LOO_se=0.0, p_LOO=41.183909481671435, shape_warn=1)

In [14]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2018_full_hierarchical_cv'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2018_full_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2018_full_hierarchical_cv'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,choice,repeat,rt,subject,trial,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,3360.0,0.0,0.0,2.0,0.762332,1.7,0.237668
1,0.0,1.0,1598.0,0.0,0.0,2.0,0.762332,1.7,0.237668
2,0.0,2.0,2172.0,0.0,0.0,2.0,0.762332,1.7,0.237668
3,0.0,3.0,2779.0,0.0,0.0,2.0,0.762332,1.7,0.237668
4,0.0,4.0,1360.0,0.0,0.0,2.0,0.762332,1.7,0.237668


### 1. no-bias GLAM

In [41]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM hierarchically...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2018_nobias_hierarchical_cv'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_FF2018_nobias_hierarchical_cv'+sufix+'.npy'))
 

Fitting no-bias GLAM hierarchically...
Generating hierarchical model for 31 subjects...


/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

Fitting 1 model(s) using NUTS...
  Fitting model 1 of 1...


/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

/!\ Automatically setting parameter precision...


In [42]:
   
# Save parameter estimates
np.save(str('results/estimates/glam_FF2018_nobias_hierarchical_cv'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

,SNR,SNR_mu,SNR_sd,b,gamma,p_error,s,t0,tau,tau_mu,tau_sd,v,v_mu,v_sd
0,177.66,144.07,31.4,1.0,1.0,0.05,0.007513,0.0,2.77,1.29,0.76,0.000048,0.000058,0.000015
1,127.76,144.07,31.4,1.0,1.0,0.05,0.010208,0.0,2.07,1.29,0.76,0.000039,0.000058,0.000015
2,155.13,144.07,31.4,1.0,1.0,0.05,0.009075,0.0,1.95,1.29,0.76,0.000057,0.000058,0.000015
3,413.19,144.07,31.4,1.0,1.0,0.05,0.006931,0.0,1.80,1.29,0.76,0.000050,0.000058,0.000015
4,91.02,144.07,31.4,1.0,1.0,0.05,0.007725,0.0,3.32,1.29,0.76,0.000090,0.000058,0.000015
5,496.86,144.07,31.4,1.0,1.0,0.05,0.007489,0.0,1.32,1.29,0.76,0.000032,0.000058,0.000015
6,105.38,144.07,31.4,1.0,1.0,0.05,0.009102,0.0,3.40,1.29,0.76,0.000086,0.000058,0.000015
7,129.34,144.07,31.4,1.0,1.0,0.05,0.006906,0.0,1.28,1.29,0.76,0.000048,0.000058,0.000015
8,136.84,144.07,31.4,1.0,1.0,0.05,0.008900,0.0,0.10,1.29,0.76,0.000062,0.000058,0.000015
9,140.92,144.07,31.4,1.0,1.0,0.05,0.007791,0.0,3.19,1.29,0.76,0.000044,0.000058,0.000015


In [30]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

,SNR,SNR_mu,SNR_sd,b,gamma,p_error,s,t0,tau,tau_mu,tau_sd,v,v_mu,v_sd
0,168.63,144.5,57.82,1.0,1.0,0.05,0.007958,0.0,2.65,0.95,0.94,0.000051,0.00006,0.000014
1,122.41,144.5,57.82,1.0,1.0,0.05,0.010110,0.0,1.78,0.95,0.94,0.000082,0.00006,0.000014
2,151.55,144.5,57.82,1.0,1.0,0.05,0.008478,0.0,1.75,0.95,0.94,0.000056,0.00006,0.000014
3,136.06,144.5,57.82,1.0,1.0,0.05,0.006783,0.0,1.11,0.95,0.94,0.000050,0.00006,0.000014
4,79.26,144.5,57.82,1.0,1.0,0.05,0.007077,0.0,2.43,0.95,0.94,0.000090,0.00006,0.000014
5,112.54,144.5,57.82,1.0,1.0,0.05,0.007223,0.0,1.58,0.95,0.94,0.000064,0.00006,0.000014
6,215.37,144.5,57.82,1.0,1.0,0.05,0.011701,0.0,1.09,0.95,0.94,0.000080,0.00006,0.000014
7,202.01,144.5,57.82,1.0,1.0,0.05,0.007848,0.0,1.25,0.95,0.94,0.000046,0.00006,0.000014
8,161.54,144.5,57.82,1.0,1.0,0.05,0.009143,0.0,0.02,0.95,0.94,0.000057,0.00006,0.000014
9,167.26,144.5,57.82,1.0,1.0,0.05,0.008773,0.0,1.47,0.95,0.94,0.000050,0.00006,0.000014


In [43]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_FF2018_nobias'+ sufix +'.npy'), glam_nobias.loo
)

/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/Users/pradyumna/anaconda/lib/python3.5/site-packages/pymc3/stats.py:292: UserWarning: Estimated shape parameter of Pareto distribution is
        greater than 0.7 for one or more samples.
       

In [44]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2018_nobias_hierarchical_cv'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_FF2018_nobias_hierarchical_cv'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_FF2018_nobias_hierarchical_cv'+sufix+'.csv'))

glam_nobias.prediction.head()

Predicting test set data using no-bias GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,choice,repeat,rt,subject,trial,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,4306.0,0.0,0.0,2.0,0.762332,1.7,0.237668
1,0.0,1.0,1850.0,0.0,0.0,2.0,0.762332,1.7,0.237668
2,0.0,2.0,2673.0,0.0,0.0,2.0,0.762332,1.7,0.237668
3,0.0,3.0,3377.0,0.0,0.0,2.0,0.762332,1.7,0.237668
4,0.0,4.0,3097.0,0.0,0.0,2.0,0.762332,1.7,0.237668


## 2. Plot fit

In [45]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

Close Figure to continue...


NameError: name 'glam_full' is not defined

## Parameters for full hierarchical model

In [ ]:
params_participant = glam_full.estimates.item(0)
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 

In [ ]:
testa = glam_nobias.prediction

In [ ]:
xlims =(0, 10)

# Compute relevant variables
df = glam.plots.add_difficulty(testa)

# Compute summary statistics
subject_means = df.groupby(['subject', 'difficulty']).rt.mean()
means = subject_means.groupby('difficulty').mean()[xlims[0]:xlims[1]]
sems = subject_means.groupby('difficulty').sem()[xlims[0]:xlims[1]]


In [ ]:
means